In [1]:
import ccxt
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta, timezone

def fetch_btc_usd_data_kraken():
    """
    Fetch the last month of BTC/USD 1-minute candles from Kraken
    Returns a pandas DataFrame with standardized UTC timestamps
    """
    try:
        # Initialize the Kraken exchange
        exchange = ccxt.kraken({
            'enableRateLimit': True,  # Important to avoid hitting rate limits
        })
        
        print(f'Exchange loaded: {exchange.id}')
        
        # Calculate timestamps for the last month (30 days) in UTC
        end_time = datetime.now(timezone.utc)
        start_time = end_time - timedelta(days=30)
        
        # Convert to millisecond timestamps
        end_timestamp = int(end_time.timestamp() * 1000)
        start_timestamp = int(start_time.timestamp() * 1000)
        
        print(f'Fetching BTC/USD data from {start_time.isoformat()} to {end_time.isoformat()}')
        
        # Check if exchange supports OHLCV data
        if not exchange.has['fetchOHLCV']:
            print('Error: Exchange does not support OHLCV data')
            return None
        
        # On Kraken, the symbol may be slightly different
        symbol = 'BTC/USD'
        # In case Kraken uses a different naming convention, you might need XBT
        if symbol not in exchange.markets:
            symbol = 'XBT/USD'
            if symbol not in exchange.markets:
                available_symbols = [s for s in exchange.markets.keys() if 'BTC' in s or 'XBT' in s]
                print(f'Symbol not found. Available BTC symbols: {available_symbols}')
                return None
        
        print(f'Using symbol: {symbol}')
        
        # Fetch all candles
        all_candles = []
        since = start_timestamp
        
        # Kraken typically limits to 720 candles per request for 1m timeframe
        while since < end_timestamp:
            print(f'Fetching batch starting from {datetime.fromtimestamp(since/1000, tz=timezone.utc).isoformat()}')
            
            # Fetch a batch of candles
            # Parameters: symbol, timeframe, since, limit
            candles = exchange.fetch_ohlcv(symbol, '1m', since, 720)
            
            if not candles or len(candles) == 0:
                print('No more candles available')
                break
            
            all_candles.extend(candles)
            print(f'Received {len(candles)} candles, total: {len(all_candles)}')
            
            # Update the since parameter for the next batch
            # The timestamp is the first element of each candle
            since = candles[-1][0] + 60000  # Add 1 minute to avoid duplicates
            
            # Add a small delay to avoid rate limiting
            time.sleep(1.5)
        
        if not all_candles:
            print('No data retrieved')
            return None
        
        # Convert to DataFrame for easier manipulation
        df = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        
        # Convert timestamp to datetime in UTC
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
        
        # Remove potential duplicates
        df = df.drop_duplicates(subset=['timestamp'])
        
        # Sort by timestamp
        df = df.sort_values('timestamp')
        
        print(f'Retrieved a total of {len(df)} candles')
        
        # Save to CSV
        csv_filename = 'btc_usd_1m_kraken.csv'
        df.to_csv(csv_filename, index=False)
        print(f'Data saved to {csv_filename}')
        
        return df
    
    except Exception as e:
        print(f'Error fetching data: {str(e)}')
        return None

# Fetch the data
df_btc = fetch_btc_usd_data_kraken()

# Display the first few rows of the DataFrame
if df_btc is not None:
    print("\nData Preview:")
    display(df_btc.head())
    
    # Show some basic statistics
    print("\nBasic Statistics:")
    print(f"Date Range: {df_btc['datetime'].min()} to {df_btc['datetime'].max()}")
    print(f"Price Range: ${df_btc['low'].min()} to ${df_btc['high'].max()}")
    print(f"Average Volume: {df_btc['volume'].mean():.2f}")
    
    # Plot the close price
    plt.figure(figsize=(14, 7))
    plt.plot(df_btc['datetime'], df_btc['close'])
    plt.title('BTC/USD Close Price (Last 30 Days)')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Exchange loaded: kraken
Fetching BTC/USD data from 2025-04-17T23:17:11.900371+00:00 to 2025-05-17T23:17:11.900371+00:00
Error fetching data: argument of type 'NoneType' is not iterable
